## Ejercicios.

Los datos provistos son datos ficticios que representan los pagos que realiza una compañía de seguros para cada uno de los siniestros reportados durante el mes de Enero del Año 2023.

1. Cargue los datos que se encuentran en la carpeta `data`, tanto el archivo de `claims.csv`, como los archivos de `claims_payments.csv`.
2. Remueva los duplicados, y conserve unicamente un único registro por `claim_id`, y este, deberá ser el registro más nuevo de la base de datos según `claim_date`, y resuelva los siguientes puntos en base a este nuevo dataset.
3. Una vez realice este proceso, deberá responder las siguientes preguntas:
   1. Cuantos siniestros distintos han ocurrido por cada uno de los días?
   2. Existen días que no tienen ningún siniestro? Cuales son estos?
   3. Cuantos registros tienen **nulos** en su campo `claim_date`?. Que porcentaje del total de la base representan?
   4. Cuál es el día que tiene **más**, y el día que tiene **menos** siniestros reportados? Que porcentaje de del total de la base representa?
   5. Cuál es la marca de vehiculos que **más** presenta siniestros, y cual es la marca que **menos** siniestros presenta?
   6. Cuál es el valor de pago más alto que ha realizado la compañia?
   7. Muestre por marca de vehículo, los valores de pagos realizados por concepto de `PÉRDIDA PARCIAL`.
   8. Cuales son los **3 primeros** tipos de pagos **más** frecuente en la compañía?
   9. Cuál es el número de días **mínimo, máximo**, y **promedio**, que han transcurrido entre un siniestro y el pago de la compañia?
   10. Cuantos siniestros aún no han recibido ningún pago de la compañia? Que porcentaje de la base representan?


4. Exporte el archivo consolidado, el cual contiene para cada `claim_id` la información de los pagos realizados por la compañia. Cada archivo se debe guardar por día, usando la siguiente nomenclatura: `payment_report_yyyy_mm_dd.csv`, donde `yyyy` hace referencia al año del siniestro, `mm` representa el mes del siniestro y `dd` representa el día del siniestro. Ejemplo: `payment_report_2023_01_05.csv`, `payment_report_2023_01_06.csv`, `...`.
5. Mueva los archivos que exportó a la carpeta `reporting`, usando Python.

## Solución

In [1]:
import pandas as pd
import numpy as np
import shutil

# Cargar datos
claims = pd.read_csv('data/claims.csv', sep=';')
payments = pd.read_csv('data/payments.csv', sep=';', encoding='latin1')

# Convertir claim_date y payment_date a tipo datetime
claims['claim_date'] = pd.to_datetime(claims['claim_date'])
payments['payment_date'] = pd.to_datetime(payments['payment_date'])

# Eliminar duplicados y conservar el registro mas nuevo por claim_id
claims = claims.sort_values(by='claim_date', ascending=False)
claims = claims.drop_duplicates(subset='claim_id', keep='first')

# Preguntas
#Cuantos siniestros distintos han ocurrido por cada uno de los dias?
siniestros_por_dia = claims['claim_date'].dt.date.value_counts()
print("Respuesta 1:\n", siniestros_por_dia)

##Existen dias que no tienen ningun siniestro? Cuales son estos?
dias_sin_siniestros = claims['claim_date'].dt.date.unique()
print("\nRespuesta 2:\n", dias_sin_siniestros)

##Cuantos registros tienen nulos en su campo claim_date?. Que porcentaje del total de la base representan?
registros_nulos = claims['claim_date'].isnull().sum()
porcentaje_nulos = (registros_nulos / len(claims)) * 100
print("\nRespuesta 3:\n", f"Registros nulos: {registros_nulos}, Porcentaje: {porcentaje_nulos:.2f}%")

##Cual es el dia que tiene mas, y el dia que tiene menos siniestros reportados? Que porcentaje de del total de la base representa?
dia_max_siniestros = siniestros_por_dia.idxmax()
dia_min_siniestros = siniestros_por_dia.idxmin()
porcentaje_max_siniestros = (siniestros_por_dia.max() / len(claims)) * 100
porcentaje_min_siniestros = (siniestros_por_dia.min() / len(claims)) * 100
print("\nRespuesta 4:\n", f"Dia con mas siniestros: {dia_max_siniestros}, Porcentaje: {porcentaje_max_siniestros:.2f}%")
print("Dia con menos siniestros: {dia_min_siniestros}, Porcentaje: {porcentaje_min_siniestros:.2f}%")

##Cual es la marca de vehiculos que mas presenta siniestros, y cual es la marca que menos siniestros presenta?
marca_mas_siniestros = claims['vehicle_brand'].value_counts().idxmax()
marca_menos_siniestros = claims['vehicle_brand'].value_counts().idxmin()
print("\nRespuesta 5:\n", f"Marca con mas siniestros: {marca_mas_siniestros}, Marca con menos siniestros: {marca_menos_siniestros}")

##Cual es el valor de pago mas alto que ha realizado la compañia?
pago_mas_alto = payments['payment_value'].max()
print("\nRespuesta 6:\n", f"Valor de pago mas alto: {pago_mas_alto}")

##Muestre por marca de vehiculo, los valores de pagos realizados por concepto de PÉRDIDA PARCIAL.
pagos_perdida_parcial = payments[payments['payment_type'] == 'PÉRDIDA PARCIAL'].merge(
    claims[['claim_id', 'vehicle_brand']], on='claim_id', how='left'
).groupby('vehicle_brand')['payment_value'].sum()
print("\nRespuesta 7:\n", pagos_perdida_parcial)

##Cuales son los 3 primeros tipos de pagos mas frecuente en la compañia?
tipos_pagos_frecuentes = payments['payment_type'].value_counts().head(3)
print("\nRespuesta 8:\n", tipos_pagos_frecuentes)

##Cual es el numero de dias minimo, maximo, y promedio, que han transcurrido entre un siniestro y el pago de la compañia?
dias_entre_siniestro_pago = (payments['payment_date'] - claims['claim_date']).dt.days
min_dias_entre_siniestro_pago = dias_entre_siniestro_pago.min()
max_dias_entre_siniestro_pago = dias_entre_siniestro_pago.max()
promedio_dias_entre_siniestro_pago = dias_entre_siniestro_pago.mean()
print("\nRespuesta 9:\n", f"Dias minimo: {min_dias_entre_siniestro_pago}, Dias maximo: {max_dias_entre_siniestro_pago}, Promedio: {promedio_dias_entre_siniestro_pago:.2f}")

##Cuantos siniestros aun no han recibido ningun pago de la compañia? Que porcentaje de la base representan?
siniestros_sin_pago = claims[~claims['claim_id'].isin(payments['claim_id'])]
porcentaje_sin_pago = (len(siniestros_sin_pago) / len(claims)) * 100
print("\nRespuesta 10:\n", f"Siniestros sin pago: {len(siniestros_sin_pago)}, Porcentaje: {porcentaje_sin_pago:.2f}%")

##Exportar archivos consolidados por dia
for date, group in payments.merge(claims[['claim_id', 'claim_date']], on='claim_id').groupby('claim_date'):
    filename = f'reporting/payment_report_{date.strftime("%Y_%m_%d")}.csv'
    group.to_csv(filename, index=False)

##Mover archivos a la carpeta reporting
shutil.move('reporting', 'reporting')


Respuesta 1:
 claim_date
2023-01-23    47
2023-01-28    42
2023-01-31    38
2023-01-03    38
2023-01-22    37
2023-01-21    37
2023-01-08    35
2023-01-30    35
2023-01-14    33
2023-01-18    33
2023-01-16    32
2023-01-26    32
2023-01-11    32
2023-01-04    31
2023-01-10    31
2023-01-02    30
2023-01-19    29
2023-01-20    29
2023-01-09    29
2023-01-07    29
2023-01-17    29
2023-01-24    28
2023-01-13    27
2023-01-12    27
2023-01-06    27
2023-01-25    26
2023-01-05    26
2023-01-27    23
2023-01-29    19
Name: count, dtype: int64

Respuesta 2:
 [datetime.date(2023, 1, 31) datetime.date(2023, 1, 30)
 datetime.date(2023, 1, 29) datetime.date(2023, 1, 28)
 datetime.date(2023, 1, 27) datetime.date(2023, 1, 26)
 datetime.date(2023, 1, 25) datetime.date(2023, 1, 24)
 datetime.date(2023, 1, 23) datetime.date(2023, 1, 22)
 datetime.date(2023, 1, 21) datetime.date(2023, 1, 20)
 datetime.date(2023, 1, 19) datetime.date(2023, 1, 18)
 datetime.date(2023, 1, 17) datetime.date(2023, 1, 16)
 